In [ ]:
import micromegas
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', None)
client = micromegas.connect()

In [ ]:
%%time
#backfill day-long partitions
now = datetime.datetime.now(datetime.timezone.utc)
begin = now - datetime.timedelta(days=10)
begin = datetime.datetime.combine(begin, datetime.time(), tzinfo=begin.tzinfo)
end = datetime.datetime.combine(now, datetime.time(), tzinfo=now.tzinfo)
size = 60*60*24
client.materialize_partitions( "blocks", begin, end, size )
client.materialize_partitions( "streams", begin, end, size )
client.materialize_partitions( "processes", begin, end, size )